# Hack & Change Changellenge 2019
### Решение задачи трека Data Science, предложенное коммандой №11 "ДуркаScience"

#### Идея состоит в следующем: 
    Для выявления наиболее эффективного территориального размещения точек выдачи мы решили предсказать 4 переменные. Основные из них: количество выданных заказов в постомате за день в среднем и бинарная переменная, определяющая, будет ли размещение постомата в данной локации выгодным или нет. Для прогнозирования мы основывались на данных тренда, количестве близлежащих университетов (акцент на целевой более активной аудитории), данных о взаимном расположении магазинов, данных об основных конкурентах, а так на исхоных данных :)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder as LE

Файл dataset.xlsx содержит частично предобработанные данные с добавленными новыми признаками

In [2]:
data = pd.read_excel("dataset.xlsx")
data.head()

,ADDRESS,CITY,REGION,Postamat_daily,cashbox_daily,Postamat_trend,cashbox_MAX,cashbox_trend,hh_500,value1,...,NEAR_Business_centers,NEAR_metro_rjd,NEAR_Stations,macro_salary_avg_yearly,PickPoint,СДЭК,Университет,Пятёрочка,DPD,Почта России
0,"г.Калуга, Литейная ул., 25/15",КАЛУГА,Калужская область,NaN,3.3,NaN,13.0,1.0,7696,10099,...,0,NaN,остановка,423321,3,1,1,1,2,0
1,"г.Калуга, Гагарина ул., 1",КАЛУГА,Калужская область,NaN,NaN,NaN,NaN,NaN,6369,10068,...,1,NaN,остановка,423321,3,0,12,14,1,0
2,"г.Калуга, Кибальчича ул., 25",КАЛУГА,Калужская область,NaN,3.1,NaN,12.0,1.0,5553,10100,...,0,NaN,NaN,423321,2,0,0,0,0,1
3,"г.Калуга, Пестеля ул., 60/49",КАЛУГА,Калужская область,3.0,2.2,0.0,12.0,1.0,7162,10099,...,0,NaN,NaN,423321,2,1,0,1,0,1
4,"г.Калуга, Ленина ул., 81",КАЛУГА,Калужская область,1.0,2.0,0.0,9.0,1.0,6269,10096,...,0,NaN,остановка,423321,4,1,6,4,1,0


#### Функция для кодирования категориальных признаков

In [3]:
def encoding(column):
    le = LE()
    le.fit(column)
    return pd.Categorical(le.transform(column))

Собираем целевые переменные в labels. Удаляем нерелевантные признаки и целевые переменные из data

In [4]:
labels = pd.DataFrame({"Postamat_daily": data['Postamat_daily'], 
                      "cashbox_daily": data['cashbox_daily']})
data = data.drop(['ADDRESS', 'Postamat_daily', 'cashbox_daily', 'cashbox_MAX',
          'macro_salary_avg_yearly'], axis = 1)

Преобразуем названия городов к одному регистру и кодируем различные кат. признаки. 

In [5]:
data['CITY'] = data['CITY'].apply(lambda x: x.lower())
data['REGION'] = encoding(data['REGION'])
data['NEAR_Stations'] = data['NEAR_Stations'].fillna("нет")
data['NEAR_Stations'] = encoding(data['NEAR_Stations'])
data = data.drop('NEAR_metro_rjd', axis = 1)
trends = data.loc[:, ['cashbox_trend', 'Postamat_trend']]
data = data.drop(['cashbox_trend', 'Postamat_trend'], axis = 1)
data['CITY'] = encoding(data['CITY'])

В дальнешем нам понадобится датасет с другими признаками, поэтому заранее создадим копию

In [6]:
second_dataset = data.copy()

In [7]:
data.head()

,CITY,REGION,hh_500,value1,value2,POPULATION,NEAR_Malls,NEAR_Business_centers,NEAR_Stations,PickPoint,СДЭК,Университет,Пятёрочка,DPD,Почта России
0,41,0,7696,10099,3382,324698,0,0,2,3,1,1,1,2,0
1,41,0,6369,10068,3236,324698,1,1,2,3,0,12,14,1,0
2,41,0,5553,10100,3436,324698,1,0,1,2,0,0,0,0,1
3,41,0,7162,10099,3257,324698,0,0,1,2,1,0,1,0,1
4,41,0,6269,10096,3195,324698,1,0,2,4,1,6,4,1,0


Также убираем данные по количеству пятерочек и университетов вокруг магазина, так как они будут нужны позже

In [8]:
data = data.drop(['Пятёрочка', 'Университет'], axis = 1)

Добавляем новые признаки из старых

In [9]:
data['population/value1'] = data['value1'] / data['POPULATION']
data['population/value2'] = data['value2'] / data['POPULATION']
data['population/hh_500'] = data['hh_500'] / data['POPULATION'] 
data['hh_500/value2'] = data['hh_500'] / data['value2']
data['hh_500/value1'] = data['hh_500'] / data['value1']
data = data.drop(['hh_500', 'value1', 'value2', 'POPULATION'], axis = 1)

Берем только те данные, для которых есть информация по среднему кол-ву заказов в кассах.

In [10]:
X = data[labels['cashbox_daily'].isnull() == False]

Для начала попробуем предсказать данные cashbox_daily

In [11]:
from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.model_selection import cross_val_score

#### MSE на кросс-валидации получился примерно равным 0.5

In [12]:
clf = RFR(n_estimators = 500, max_depth = 20, random_state = 17)
-cross_val_score(clf, X,  labels['cashbox_daily'].dropna(), cv=10, scoring='neg_mean_squared_error').mean()

0.4875415749666672

Попробуем с помощью этой модели предсказать те данные, в которых у нас нет cashbox_daily

In [13]:
clf = RFR(n_estimators = 500, max_depth = 20, random_state = 17)
clf.fit(X, labels['cashbox_daily'].dropna());

In [14]:
preds = clf.predict(data[labels['cashbox_daily'].isnull() == True])
counter = 0
for i in np.arange(len(labels)):
    if np.isnan(labels['cashbox_daily'][i]):
        labels['cashbox_daily'][i] = preds[counter]
        counter += 1

## CASHBOX TREND PREDICTION

Заполняем еще одну таблицу

In [15]:
second_dataset = second_dataset.drop(['Пятёрочка', 'Университет'], axis = 1)
second_dataset['value1/population'] = second_dataset['value1'] / second_dataset['POPULATION']
second_dataset['value2/population'] = second_dataset['value2'] / second_dataset['POPULATION']
second_dataset['hh_500/population'] = second_dataset['hh_500'] / second_dataset['POPULATION'] 
second_dataset['hh_500/value2'] = second_dataset['hh_500'] / second_dataset['value2']
second_dataset['hh_500/value1'] = second_dataset['hh_500'] / second_dataset['value1']
second_dataset = second_dataset.drop(['hh_500', 'value1', 'value2', 'POPULATION'], axis = 1)

In [16]:
second_dataset['cashbox_daily'] = labels['cashbox_daily']

In [17]:
second_dataset.head()

,CITY,REGION,NEAR_Malls,NEAR_Business_centers,NEAR_Stations,PickPoint,СДЭК,DPD,Почта России,value1/population,value2/population,hh_500/population,hh_500/value2,hh_500/value1,cashbox_daily
0,41,0,0,0,2,3,1,2,0,0.031103,0.010416,0.023702,2.275577,0.762056,3.3000
1,41,0,1,1,2,3,0,1,0,0.031007,0.009966,0.019615,1.968171,0.632598,2.4088
2,41,0,1,0,1,2,0,0,1,0.031106,0.010582,0.017102,1.616123,0.549802,3.1000
3,41,0,0,0,1,2,1,0,1,0.031103,0.010031,0.022057,2.198956,0.709179,2.2000
4,41,0,1,0,2,4,1,1,0,0.031094,0.009840,0.019307,1.962128,0.620939,2.0000


In [18]:
with_labels = second_dataset[trends['cashbox_trend'].isnull() == False]

Опять будем предсказывать с помощью random forest

In [19]:
from sklearn.ensemble import RandomForestClassifier

 #### В качестве метрики мы выбрали precision, так как, если бизнес откроет точку, а она не окупится, будет хуже, чем, если не открывать точку. 
    Она вышла 91%

In [20]:
rfc = RandomForestClassifier(max_depth = 20, n_estimators = 500, random_state = 17)
cross_val_score(rfc, with_labels, trends['cashbox_trend'].dropna(), cv = 10,
               scoring = 'precision').mean()

0.9093040293040293

Предскажем для всего датасета

In [21]:
rfc.fit(with_labels, trends['cashbox_trend'].dropna())

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=20, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=17, verbose=0,
                       warm_start=False)

In [22]:
preds = rfc.predict(second_dataset[trends['cashbox_trend'].isnull() == True])
counter = 0
for i in np.arange(len(trends)):
    if np.isnan(trends['cashbox_trend'][i]):
        trends['cashbox_trend'][i] = preds[counter]
        counter += 1

In [23]:
second_dataset['Университет'] = pd.read_excel("dataset.xlsx")['Университет']
second_dataset['Пятёрочка'] = pd.read_excel("dataset.xlsx")['Пятёрочка']

## POSTOMAT DAILY PREDICTION
#### Для предсказания данного признака MSE = ~1.5

In [24]:
with_labels = second_dataset[trends['Postamat_trend'].isnull() == False]

In [25]:
from sklearn.ensemble import RandomForestRegressor

In [26]:
rfc = RandomForestRegressor(max_depth = 20, n_estimators = 500, random_state = 17)
-cross_val_score(rfc, with_labels, labels['Postamat_daily'].dropna(), cv = 10,
               scoring = 'neg_mean_squared_error').mean()

1.5541911857142856

In [27]:
label = labels['Postamat_daily'].dropna()

In [28]:
rfc.fit(with_labels, label)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=20,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=500,
                      n_jobs=None, oob_score=False, random_state=17, verbose=0,
                      warm_start=False)

In [29]:
y_pred = rfc.predict(second_dataset[trends['Postamat_trend'].isnull() == True])
counter = 0
for i in np.arange(len(trends)):
    if np.isnan(labels['Postamat_daily'][i]):
        labels['Postamat_daily'][i] = y_pred[counter]
        counter += 1

In [30]:
labels

,Postamat_daily,cashbox_daily
0,2.858,3.3000
1,2.586,2.4088
2,3.478,3.1000
3,3.000,2.2000
4,1.000,2.0000
...,...,...
412,1.640,0.6248
413,1.442,0.5014
414,1.640,0.4654
415,1.468,0.5654


## Теперь попробуем предсказывать Postamat trend
#### На перекрестной проверке для признака Postmat_trend точность предсказания 60-65%

In [31]:
rf = RandomForestClassifier(max_depth = 2, n_estimators = 500)
cross_val_score(rf, with_labels, trends['Postamat_trend'].dropna(), cv = 10,
               scoring = 'accuracy').mean()

0.6178571428571429

In [32]:
rf.fit(with_labels, trends['Postamat_trend'].dropna())
y_pred = rf.predict(second_dataset[trends['Postamat_trend'].isna()])

In [33]:
counter = 0
for i in np.arange(len(trends)):
    if np.isnan(trends['Postamat_trend'][i]):
        trends['Postamat_trend'][i] = y_pred[counter]
        counter += 1

## Таблица с итоговыми предсказаниями по адресам

In [34]:
addresses = pd.read_excel('dataset.xlsx')['ADDRESS']
submiss = pd.DataFrame({'Адреса': addresses, 'Тренд развития почтомата при открытии': trends['cashbox_trend'], 'Тренд развития выдачи на кассе при открытии': trends['Postamat_trend'], 'Прогнозируемое количество заказов на кассе в день': labels['cashbox_daily'], 'Прогнозируемое количество заказов на почтомате в день': labels['Postamat_daily']})
submiss

,Адреса,Тренд развития почтомата при открытии,Тренд развития выдачи на кассе при открытии,Прогнозируемое количество заказов на кассе в день,Прогнозируемое количество заказов на почтомате в день
0,"г.Калуга, Литейная ул., 25/15",1.0,1.0,3.3000,2.858
1,"г.Калуга, Гагарина ул., 1",1.0,0.0,2.4088,2.586
2,"г.Калуга, Кибальчича ул., 25",1.0,0.0,3.1000,3.478
3,"г.Калуга, Пестеля ул., 60/49",1.0,0.0,2.2000,3.000
4,"г.Калуга, Ленина ул., 81",1.0,0.0,2.0000,1.000
...,...,...,...,...,...
412,д.Чернятино86А,0.0,0.0,0.6248,1.640
413,"п.Дубовка, квартал 5/15, Центральный перс.6",1.0,0.0,0.5014,1.442
414,"г.Донской, мкр.Северо-Задонск, Мичурина ул, 76...",0.0,0.0,0.4654,1.640
415,"с.Гремячее, Новики ул., 12",1.0,0.0,0.5654,1.468
